<a href="https://colab.research.google.com/github/TA-aiacademy/course_3.0/blob/v2-5_nlp/09_v2-5_NLP/Part5/01_Bert_finetune_glue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transformer Pre-trained model

這一章節介紹目前自然語言處理最強大的模型-`Transformer`，`Transformer`相較於`RNN`系列的模型，`Transformer`在表現(`metrics`)以及計算效率(`parallel`)都有絕對的優勢，著名的`pre-train`模型如下，連結為各個模型的論文路徑，基本上這些模型都是`Transformer`的變形，不同的地方在於預訓練的策略，例如資料量大小、`Masked`的差異以及`Self-attention`矩陣的差異，最特別的是最後一個`ELECTRA`，是在`2019`年`11`月初提出的論文，結合了`transformer`還有`GAN`。

* BERT: https://arxiv.org/abs/1810.04805
 - Masked Language Modeling + Next Sentence Prediction


* GPT: https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf
 - AutoRegressive Prediction


* Transformer-XL: https://arxiv.org/abs/1901.02860
 - Learning dependency beyond a fixed length(>512)


* XLNet: https://arxiv.org/abs/1906.08237
 - Permutation Modeling


* XLM: https://arxiv.org/abs/1901.07291
 - Pretrain on cross-lingual language


* RoBERTa: https://arxiv.org/abs/1907.11692
 - Pretrain model longer, more data


* DistilBERT: https://arxiv.org/abs/1910.01108
* CTRL: https://arxiv.org/abs/1909.05858
* ELECTRA: https://openreview.net/pdf?id=r1xMH1BtvB
 - Transformer + GAN


### [GLUE Benchmark](https://gluebenchmark.com/leaderboard)

## [Transformer](https://huggingface.co/transformers/)

這邊我們使用`Transformers`套件來進行`finetune`，在進行`finetune`之前，需要了解自然語言處理任務上的差異，最主要分為兩種分類任務：

1. `Text classification`: 輸入一個句子，輸出該句子的分類。
2. `Sentence-Pair classification`: 輸入兩個句子的pair，輸出兩個句子之間的關係。

* PS. 這些預訓練模型除了表現亮眼之外，最重要的貢獻在於預訓練後的`word embedding`，`word embedding`表示在文本中，詞與詞之間的關係，最著名的例子就是: 男性 - 女性 = 國王 - 皇后，像這樣的對應關係，有訓練良好的`word embedding`基本上在其他應用任務表現也會不錯，例如聊天機器人。

在這裡我們會使用`BERT`來進行`finetune`。

In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import os
from sklearn.metrics import classification_report, confusion_matrix

from transformers import *

## 模型名稱解釋

* `bert-base-uncased`:
  - `bert`: 模型名稱
  - `base`: 模型大小，`base`表示層數為$12$層, `word embedding(hidden)`為$768$維, `heads`為$12$，另外有`large`，層數為$24$層，`word embedding(hidden)`為$1024$維，`heads`為$16$。
  - `uncased`: 表示對於文本的前處理，`uncased`表示字全部轉小寫，反之`cased`表示維持原樣。

另外不只有這些模型，其餘模型可以參考：
https://huggingface.co/transformers/pretrained_models.html

In [ ]:
"""
載入預訓練模型
"""
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

In [ ]:
"""
載入模型斷詞工具
"""
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

## Finetune

![img](https://hackmd.io/_uploads/rJIRRucJp.png)

所有預訓練模型都是在[GLUE Benchmark](https://gluebenchmark.com/leaderboard)進行競賽，這個競賽提供多種不同的自然語言處理任務，這些任務都是屬於分類任務，只是差別在於資料集大小以及來源而已，這裡我們使用其中一種分類任務`MRPC`來進行`finetune`。

* 資料來源: [tensorflow dataset](https://www.tensorflow.org/datasets/catalog/overview#wmt19_translate)

In [ ]:
data, info = tfds.load('glue/mrpc', with_info=True)

### Info

資料集的介紹，最需要注意的地方就是資料集的樣子，因為`MRPC`是屬於`Sentnece-Pair classification`任務，所以資料集包括了`sentence1`和`sentence2`對應一個`label`，`MRPC`主要是在分類兩個句子之間的語義是否相同，`label`為$1$表示相同，反之$0$表示不同。

因為是競賽資料集，所以資料集已經切割好為train, validation以及test。

In [ ]:
info

In [ ]:
for k, v in data.items():
    print('key:', k)
    print('data shapes:\n', v)
    print('-' * 20)

### Dataset overview

`tensorflow`儲存資料的方式都是以`tf.data.Data`型態來儲存，可以使用`iter`來建立`generator`，並使用`next`來觀看第一筆資料，資料中包含了`idx`、`label`、`sentence1`以及`sentence2`。

In [ ]:
assert isinstance(data['train'], tf.data.Dataset)

temp = data['train']
temp_gen = iter(temp)
next(temp_gen)

### Training data format

接下來我們需要將資料集轉換成模型可讀取的格式，輸入格式有三個：

* `input_ids`: 這表示句子斷完詞之後轉成`token embeddings`，每一個詞有一個`id`，如下圖，其中`101`表示`[CLS]`，`102`表示`[SEP]`，因為`MPRC`是`Sentence-Pair classification`任務，所以下面的範例中會看到兩個`102`。

![](https://hackmd.io/_uploads/Hyl-1F5ka.png)

* `attention mask`: 因為`Transformer`會限制輸入句子的長度，最大限制為`512`，而我們選擇`128`，但不是所有的句子長度都是128，所以需要在後面進行`padding`(就是補0)，最主要的目的是不去計算`padding`位置的`loss`。

* `token_type_ids`: 用來表示`Segment embedding`，如上圖，表示詞屬於哪一個句子，因為`MRPC`有兩個句子，所以`ids`有2種，`0`和`1`。

In [ ]:
max_length = 128
task = 'mrpc'

train_dataset = glue_convert_examples_to_features(data['train'],
                                                  tokenizer,
                                                  max_length,
                                                  task)
valid_dataset = glue_convert_examples_to_features(data['validation'],
                                                  tokenizer,
                                                  max_length,
                                                  task)
test_dataset = glue_convert_examples_to_features(data['test'],
                                                 tokenizer,
                                                 max_length,
                                                 task)

### Example

觀察轉換過後的資料集。

In [ ]:
next(iter(train_dataset))

### Parameter settings

在`tf.data.Dataset`中，通常會在訓練資料集後面接上三個標準的操作：

* `.shuffle()`: 打亂資料集的方式，會先從資料集中隨機抽取`buffer_size`筆資料進去`buffer`，然後再`buffer`從中抽取`batch_size`筆資料進行訓練，丟進`buffer`的步驟主要是在處理無法一次將所有資料集丟進記憶體進行訓練的情形。

* `.batch()`: 每次迭代使用的資料數量。
* `.repeat()`: `epochs`數量。

In [ ]:
buffer_size = 100
train_bz = 16
epochs = 3
valid_bz = 50

train_dataset = train_dataset.shuffle(buffer_size).batch(train_bz).repeat(epochs)
valid_dataset = valid_dataset.batch(valid_bz)
test_dataset = test_dataset.batch(valid_bz)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5,
                                     epsilon=1e-8,
                                     clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                     reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

## Training

* `.fit()`: 支援`generator`的輸入方式，也可以用`fit_generator`。
* `steps_per_epoch`: 每個`epoch`訓練幾次，通常是$\frac{train\_size}{batch\_size}$，遍歷整個訓練集。
* `validation_steps`: 與`steps_per_epoch`同義。

In [ ]:
history = model.fit(train_dataset,
                    epochs=epochs,
                    steps_per_epoch=3668//train_bz,
                    validation_data=valid_dataset,
                    validation_steps=408//valid_bz)

## Evaluation

In [ ]:
valid_pred = model.predict(valid_dataset)
valid_pred_ids = np.argmax(valid_pred.logits, axis=-1)

In [ ]:
valid_pred_ids

In [ ]:
"""
從tf.data.Dataset中拿取label
"""
valid_label = list()
for x in valid_dataset:
    valid_label += x[1].numpy().tolist()

In [ ]:
print(classification_report(y_pred=valid_pred_ids, y_true=valid_label))

In [ ]:
confm = confusion_matrix(y_pred=valid_pred_ids, y_true=valid_label)

index = ['Actual_0', 'Actual_1']
columns = ['Pred_0', 'Pred_1']
pd.DataFrame(confm, index=index, columns=columns)

## Save model

In [ ]:
save_path = 'save_glue'
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [ ]:
model.save_pretrained(save_path)

## Load model and predict

這邊參考`MRPC`的輸入格式，一樣會使用`glue_convert_examples_to_features`這個函數進行轉換。

In [ ]:
new_model = TFBertForSequenceClassification.from_pretrained(save_path)

In [ ]:
sentence1 = ["Anorld Schwarzenegger is my idol."]
sentence2 = ["My favorite idol is Anorld Schwarzenegger."]

test_dataset = pd.DataFrame(dict(idx=list(range(len(sentence1))),
                                 label=[0]*len(sentence1),
                                 sentence1=sentence1,
                                 sentence2=sentence2))

In [ ]:
"""
模仿GLUE的輸入格式: (idx, label, sentence1, sentence2)
其中label是假的，是因為輸入需要，不會影響預測值
"""
test_dataset

In [ ]:
test_gen = tf.data.Dataset.from_tensor_slices(dict(test_dataset))

In [ ]:
test_gen = glue_convert_examples_to_features(test_gen, tokenizer, max_length, task)

In [ ]:
test_gen = test_gen.batch(1)

In [ ]:
next(iter(test_gen))

In [ ]:
pred = new_model.predict(test_gen)

In [ ]:
pred

In [ ]:
pred_ids = np.argmax(pred.logits, axis=-1)

In [ ]:
print(pred_ids[0])